<a href="https://colab.research.google.com/github/DhruvArora96/nfl_kaggle_competition-/blob/main/nfl_compeition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 4.2MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-cp36-none-any.whl size=73269 sha256=b269ca1c07bcf063621b85e831d761678fca6dd389c02b62f327df114515d99d
  Stored in directory: /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dhruvarora96","key":"193f6f8453f255dfe63703edb500f099"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import seaborn as sns
import cv2
from tqdm import tqdm
import torch.utils.data
import torch
from torchvision import transforms, utils

In [21]:
train_df = pd.read_csv('train_labels.csv')
train_df = train_df.fillna(0)

In [23]:
image_names =[]
for i in tqdm(range(len(train_df))):
    image_names.append(train_df.video[i].replace('.mp4','_frame')+str(train_df.frame[i])+'.jpg')
train_df['imageNames'] = image_names

100%|██████████| 983885/983885 [00:21<00:00, 45391.52it/s]


In [24]:
frame_array = [-3,-2,-1,0,1,2,3]
impact_only_training_df = train_df[train_df['impact']==1]
for row in tqdm(impact_only_training_df[['video','frame']].values):
    train_df.loc[(train_df['video']==row[0])
                & (train_df['frame'].isin(row[1]+np.array(frame_array))),
                'impact']=1

100%|██████████| 2291/2291 [03:06<00:00, 12.26it/s]


In [34]:
video_path = 'train/'
image_path = 'train_images/'
!mkdir train_images

mkdir: cannot create directory ‘train_images’: File exists


In [25]:
images_list = train_df[train_df['impact']==1]
images_list=images_list[['video','frame']].drop_duplicates()
images_list=images_list.reset_index()
images_list=images_list.drop('index',axis=1)

In [35]:
def make_images(image_path,video_path,video_name,frame):
    vidcap = cv2.VideoCapture(video_path+video_name)
    success,_ = vidcap.read() 
    if success:
        vidcap.set(1,frame)
        _,image = vidcap.read()
        image_name = str(video_name).replace('.mp4','_frame')+str(frame)+'.jpg'
        cv2.imwrite(image_path+f'{image_name}',image)

In [36]:
for i in tqdm(range(len(images_list))):
    make_images(image_path,video_path,images_list['video'][i],images_list['frame'][i])

100%|██████████| 8304/8304 [1:28:53<00:00,  1.56it/s]


In [28]:
train_df['x0']=train_df['left']
train_df['y0']=train_df['top']
train_df['x1']=train_df['left']+train_df['width']
train_df['y1']=train_df['top']+train_df['height']

In [ ]:
class NFLDataset(torch.utils.data.Dataset):
    def __init__(self,root,transforms=None,ids):
        self.root = root
        self.transforms = transforms
        self.idx = ids
    def __getitem__(self,idx):
        target ={}
        image_name = images_list[idx]
        target['image']=cv2.imread(image_path+str(image_name))
        boxes = train_df.loc[(train_df['imageNames']==image_name),['x0','y0','x1','y1']]
        target['boxes']=boxes
        target['image_id']=idx

In [54]:
images_list

,video,frame
0,57583_000082_Endzone.mp4,35
1,57583_000082_Endzone.mp4,36
2,57583_000082_Endzone.mp4,37
3,57583_000082_Endzone.mp4,38
4,57583_000082_Endzone.mp4,39
...,...,...
8299,58107_004362_Sideline.mp4,440
8300,58107_004362_Sideline.mp4,441
8301,58107_004362_Sideline.mp4,442
8302,58107_004362_Sideline.mp4,443
